# Train Example

Consider a set of 6 trains named {A,B,C,D,E, F} with the following set of temporal constraints.
1. A, B and E reach the platform at the same time
2. A leaves before B.
3. A leaves after or at the same time as C but before the arrival of D.
4. D and F arrive at the same time as B is leaving.
5. E and D leave at the same time.

The problem is to find the minimum number of tracks required at the station. Because of security reasons, we do not allow that a train to arrive on a track from which a train is currently leaving.

## Solution using superposition

Let us represent all the trains using a, b, c, d, e, f and 'la' represents arrival of train at the platform and 'ra' represents leaving of train from the platform and so on.

The constraints can be represented as follows:

`E1 = [{'la','lb','le'}]
E2 = [{'ra'},{'rb'}]
E3 = [{'rc'},{'ra'},{'ld'}] or [{'rc','ra'},{'ld'}]
E4 = [{'ld','lf','rb'}]
E5 = [{'re','rd'}]` 

To solve this using the superposition theorem, we will have to add a few more constraints for ensuring that the left border occurs befor the right border.

`E6 = [{'la'},{'ra'}]
E7 = [{'lb'},{'rb'}]
E8 = [{'lc'},{'rc'}]
E9 = [{'ld'},{'rd'}]
E10 = [{'le'},{'re'}]
E11 = [{'lf'},{'rf'}]`

In [7]:
'''
Superposition implementation on the given two strings
'''
def superpose(string1, string2, voc1, voc2):
    if len(string1) == len(string2) == 0:
        return [string1]
    else:
        temp = []
        if len(string1) > 0:
            h1 = string1[0]
            t1 = string1[1:]
            h1v2 = h1.intersection(voc2)
            if len(h1v2) == 0:
                for res in superpose(t1,string2,voc1,voc2):
                    temp.append([h1] + res)     
            if len(string2) > 0:
                h2 = string2[0]
                t2 = string2[1:]
                h3 = h1.union(h2)
                if h1v2.issubset(h2):
                    h2v1 = h2.intersection(voc1)
                    if h2v1.issubset(h1):
                        for res in superpose(t1,t2,voc1,voc2):
                            temp.append([h3] + res)
        if len(string2) > 0:
            h2 = string2[0]
            t2 = string2[1:]
            h2v1 = h2.intersection(voc1)
            if len(h2v1) == 0:
                for res in superpose(string1,t2,voc1,voc2):
                    temp.append([h2] + res)
        return temp

def super(s1,s2):
    return superpose(s1,s2,voc(s1),voc(s2))

def voc(string): 
    if len(string) == 0:
        return {}
    else:
        return string[0].union(voc(string[1:]))

def proj(string,set): 
    if len(string) == 0:
        return string
    else:
        h1 = string[0].intersection(set)
        if len(h1) == 0:
            return  proj(string[1:],set)
        else:
            return [h1] + proj(string[1:],set)
        
'''
Implementation to find the highest number of tracks required by each string solution
'''
def get_track_key(tracks, to_find):
    for k,v in tracks.items():
        if v == to_find:
            return k

def get_tracks(s):
    tracks = {
        'track1':'',
        'track2':'',
        'track3':'',
        'track4':'',
        'track5':'',
        'track6':'',
    }
    unique_tracks = set()
    to_empty = False
    empty_loc = []
    for s1 in s:
        for s2 in s1:
            if s2[0] == 'l':
                loc = get_track_key(tracks, '')
                tracks[loc] = s2[1]
                unique_tracks.add(loc)
            else:
                empty_loc.append(get_track_key(tracks, s2[1]))
                to_empty = True
        if to_empty:
            for e in empty_loc:
                tracks[e] = ''
    return len(unique_tracks)


As there are two cases for E3, we solve by taking one at a time into consideration:

In [8]:
E1 = [{'la','lb','le'}]
E2 = [{'ra'},{'rb'}]
E3 = [{'rc'},{'ra'},{'ld'}]
E4 = [{'ld','lf','rb'}]
E5 = [{'re','rd'}]

E6 = [{'la'},{'ra'}]
E7 = [{'lb'},{'rb'}]
E8 = [{'lc'},{'rc'}]
E9 = [{'ld'},{'rd'}]
E10 = [{'le'},{'re'}]
E11 = [{'lf'},{'rf'}]

E = [E2,E3,E4,E5,E6,E7,E8,E9,E10,E11]
previous_satisfied_strings = [E1]

for e in E:
    satisfied_strings = []
    for strings in previous_satisfied_strings:
        for s in super(strings, e):
            if s not in satisfied_strings:
                satisfied_strings.append(s)
    previous_satisfied_strings = satisfied_strings
print("No.", "\t", "String s", "\t\t\t\t\t\t\t\t\t\t    ", "len(s)", " ","Min tracks")
for i,s in enumerate(satisfied_strings):
    if len(s) == 7:
        print(i+1, "\t", s, "\t", len(s), "\t",get_tracks(s))
    else:
        print(i+1, "\t", s, "\t\t", len(s), "\t",get_tracks(s))

No. 	 String s 										     len(s)   Min tracks
1 	 [{'la', 'le', 'lb'}, {'lc'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'re', 'rd'}, {'rf'}] 	 7 	 4
2 	 [{'la', 'le', 'lb'}, {'lc'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'re', 'rf', 'rd'}] 		 6 	 4
3 	 [{'la', 'le', 'lb'}, {'lc'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'rf'}, {'re', 'rd'}] 	 7 	 4
4 	 [{'le', 'lb', 'la', 'lc'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'re', 'rd'}, {'rf'}] 		 6 	 4
5 	 [{'le', 'lb', 'la', 'lc'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'re', 'rf', 'rd'}] 		 5 	 4
6 	 [{'le', 'lb', 'la', 'lc'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'rf'}, {'re', 'rd'}] 		 6 	 4
7 	 [{'lc'}, {'la', 'le', 'lb'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'re', 'rd'}, {'rf'}] 	 7 	 4
8 	 [{'lc'}, {'la', 'le', 'lb'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'re', 'rf', 'rd'}] 		 6 	 4
9 	 [{'lc'}, {'la', 'le', 'lb'}, {'rc'}, {'ra'}, {'lf', 'rb', 'ld'}, {'rf'}, {'re', 'rd'}] 	 7 	 4
10 	 [{'lc'}, {'le', 'lb', 'la', 'rc'}, {'ra'}, {'lf', 'rb', 'ld'}

In [9]:

E1 = [{'la','lb','le'}]
E2 = [{'ra'},{'rb'}]
E3 = [{'rc','ra'},{'ld'}]
E4 = [{'ld','lf','rb'}]
E5 = [{'re','rd'}]


E6 = [{'la'},{'ra'}]
E7 = [{'lb'},{'rb'}]
E8 = [{'lc'},{'rc'}]
E9 = [{'ld'},{'rd'}]
E10 = [{'le'},{'re'}]
E11 = [{'lf'},{'rf'}]

E = [E2,E3,E4,E5,E6,E7,E8,E9,E10,E11]
previous_satisfied_strings = [E1]

for e in E:
    satisfied_strings = []
    for strings in previous_satisfied_strings:
        for s in super(strings, e):
            if s not in satisfied_strings:
                satisfied_strings.append(s)
    previous_satisfied_strings = satisfied_strings
print("No.", "\t", "String s", "\t\t\t\t\t\t\t\t\t\t    ", "len(s)", " ","Min tracks")
for i,s in enumerate(satisfied_strings):
    print(i+1, "\t", s, "\t\t", len(s), "\t",get_tracks(s))

No. 	 String s 										     len(s)   Min tracks
1 	 [{'la', 'le', 'lb'}, {'lc'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'re', 'rd'}, {'rf'}] 		 6 	 4
2 	 [{'la', 'le', 'lb'}, {'lc'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'re', 'rf', 'rd'}] 		 5 	 4
3 	 [{'la', 'le', 'lb'}, {'lc'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'rf'}, {'re', 'rd'}] 		 6 	 4
4 	 [{'le', 'lb', 'la', 'lc'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'re', 'rd'}, {'rf'}] 		 5 	 4
5 	 [{'le', 'lb', 'la', 'lc'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'re', 'rf', 'rd'}] 		 4 	 4
6 	 [{'le', 'lb', 'la', 'lc'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'rf'}, {'re', 'rd'}] 		 5 	 4
7 	 [{'lc'}, {'la', 'le', 'lb'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'re', 'rd'}, {'rf'}] 		 6 	 4
8 	 [{'lc'}, {'la', 'le', 'lb'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'re', 'rf', 'rd'}] 		 5 	 4
9 	 [{'lc'}, {'la', 'le', 'lb'}, {'ra', 'rc'}, {'lf', 'rb', 'ld'}, {'rf'}, {'re', 'rd'}] 		 6 	 4


### Answer

After solving the above problem using superposition, we get a total of 24 solution strings (15 + 9 for each of the two possibilities of E3) of length between 4 and 7. We can see that the minimum number of tracks required for each of the case is 4. Hence, the answer to the problem is 4.

## Solution using Mix and Join operation in the paper

The alphabet is = {a, b, c, d, e, f} and the constraints are given as follows:

`E1 = [{a,b,e}] . ([{a}] X [{b}] X [{e}])
E2 = ([{a}] X [{b}]) . [{a}{b}]
E3 = (([{a}] X [{c}{c}]) . [{a}{d}{d}]) U (([{a}] X [{c}]) . [{a,c}{d}{d}])
E4 = [{b}{b,d,f}] . ([{f}] X [{d}])
E5 = ([{e}] X [{d}]) . [{d,e}] `

For solving all the constraints the join is computed as: 

`E = E1 J E2 J E3 J E4 J E5`

The simplified the S-expression evaluates to an S-language containing 24 S-words of length between 5 and 7. The final language is written as

`E = ([{c}{c}] X [{a,b,e}]).[{a}].[{b,d,f}].([{f}] X [{e,d}]) U
    ([{a,b,e}] X [{c}]).[{a,c}].[{b,d,f}].([{f}] X [{e,d}])`
    
First, C stops and leaves, then A, B, E arrive all at the same time, then we need at least three tracks. But A leaves only before the arrival of D and F, then we need one more track. The answer of the problem is then that 4 tracks are enough.

## Comparison

### 1. Representation of constraints

The constraints give the information about either the arrival of the train or the departure of the trains. In the case of border strings, it is easier to represent them as they are differentiated as 'la' and 'ra'. Whereas, in the case of s-languages, it requires the occurence of two 'a' to represent the arrival and the departure respectively which gives rise to more possibilities. 

For instance, the 5th constraint - E and D leave at the same time can be expressed as : 

[{'re','rd'}] in the case of border strings.

and ([{e}] X [{d}]) . [{d,e}] in the case of s-words which result in three strings - 
1. [{e}], [{d}], [{d,e}]
2. [{e}, {d}], [{d,e}]
3. [{d}], [{e}], [{d,e}] 

Here the X represents the uncertainity of the arrival of e and d. Although this mix operation (X) provides a shorter way of representation, we need to add strings which we do not have information on which adds additional complexity.

### 2. Additional constraints for border strings

In the case of border strings few more constraints needs to be added to make sure that the left border ('la') occurs strictly before the right border ('ra').

Eg. E6 = [{'la'},{'ra'}], E7 = [{'lb'},{'rb'}] etc.

### 3. Solving the constraints

In both the cases, solving the constraints give rise to 24 strings.